In [9]:
!pip install spotipy

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd
import time
import datetime
import os

client_id = "71a6172ac25b4574967c90eb0a36929b"
client_secret = "f54ac611844c4d05a39f521208eaed84"


client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
# 검토 완료 진행상황 표시

start_time = time.time()

temporary_df = pd.DataFrame()  
temporary_df2 = pd.DataFrame()  
temporary_df3 = pd.DataFrame() 

artist_cache = {}  # 아티스트 정보 캐시 생성 // 요청 제한을 피하기 위해

year_list = ['2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010','2009', '2008', '2007', '2006', '2005', '2004', '2003', '2002', '2001', '2000']
genre_list = ['k-pop', 'acoustic', 'club', 'dance', 'edm', 'disco', 'electronic', 'happy', 'hip-hop','party', 'pop', 'r-n-b', 'rock', 'romance', 'sleep', 'soul']


total_workload = len(year_list) * len(genre_list) * 100  
completed_workload = 0

for year in year_list:
    for genre in genre_list:
        print(f"{year}_{genre}: 다운로드")    
        
        artist_name = []
        track_name = []
        track_popularity = []
        artist_id = []
        track_id = []  
        release_date = []
        album_image = []

        artist_popularity = []
        artist_followers =[]

        track_features_list = []
        
        for i in range(0, 1000, 50):
            track_results = sp.search(q=f'year:{year} genre:{genre}', type='track', limit=50, offset=i)
            for i, t in enumerate(track_results['tracks']['items']):
                artist_id.append(t['artists'][0]['id'])
                artist_name.append(t['artists'][0]['name'])
                track_id.append(t['id'])
                track_name.append(t['name'])
                release_date.append(t['album']['release_date'])
                track_popularity.append(t['popularity'])
                try:
                    album_image.append(t['album']['images'][1]['url'])
                except IndexError:
                    album_image.append(None)                
                print(f"음악트랙 데이터 수집 횟수 : {i+1} // 3초 대기")
                time.sleep(3)

                # 아티스트 정보를 요청 코드
                artist_info = None
                if t['artists'][0]['id'] in artist_cache:
                    artist_info = artist_cache[t['artists'][0]['id']]
                else:
                    artist_info = sp.artist(t['artists'][0]['id'])
                    artist_cache[t['artists'][0]['id']] = artist_info

                artist_popularity.append(artist_info['popularity'])  
                artist_followers.append(artist_info['followers']['total'])  
                print(f"아티스트 데이터 수집 횟수 : {i+1} // 3초 대기")
                time.sleep(3)

                # 오디오 특성 요청 코드
                track_features = sp.audio_features([t['id']])
                if track_features:
                    track_features_list.append(track_features[0])
                else:
                    track_features_list.append(None)
                print(f"음악특성 데이터 수집 횟수: {i+1} // 3초 대기")
                time.sleep(3)
                
                completed_workload += 1  
                progress = (completed_workload / total_workload) * 100  
                print(f"전체 진행 상황: {progress:.2f}%")
                
        track_df = pd.DataFrame({ 'artist_id': artist_id, 'artist_name': artist_name, 'track_id': track_id, 'track_name': track_name, 
                                 'release_date': release_date, 'track_popularity': track_popularity, 'album_image': album_image})

        track_df2 = pd.DataFrame({'artist_id': artist_id, 'artist_name': artist_name, 'artist_popularity': artist_popularity, 'artist_followers': artist_followers,
                                  'track_id': track_id})


        track_df3 = pd.DataFrame(track_features_list)
        
        temporary_df = pd.concat([temporary_df, track_df])
        temporary_df2 = pd.concat([temporary_df2, track_df2])  
        temporary_df3 = pd.concat([temporary_df3, track_df3])

temporary_df.drop_duplicates(subset='track_id', inplace=True)

timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
#temporary_df.to_csv(f'/home/ubuntu/project/4.5HZ/Data/test/test_track.csv', index=False)
temporary_df.to_csv(f'/home/ubuntu/project/4.5HZ/Data/test/API/track/{year}_{genre}_track_{timestamp}.csv', index=False)


#temporary_df2.drop_duplicates(subset='artist_id', inplace=True)
temporary_df2.drop_duplicates(subset='track_id', inplace=True)
temporary_df2 = temporary_df2.drop(columns=['track_id'])

timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
#temporary_df2.to_csv(f'/home/ubuntu/project/4.5HZ/Data/test/test_artist.csv', index=False)
temporary_df2.to_csv(f'/home/ubuntu/project/4.5HZ/Data/test/API/artist/{year}_{genre}_artist_{timestamp}.csv', index=False)

temporary_df3.drop_duplicates(subset='id', inplace=True)
temporary_df3 = temporary_df3.rename(columns={'id': 'track_id'})
temporary_df3 = temporary_df3.rename(columns={'key': 'music_key'})
temporary_df3 = temporary_df3.rename(columns={'mode': 'music_mode'})
temporary_df3 = temporary_df3.drop(columns=['type', 'uri', 'track_href', 'analysis_url', 'duration_ms'])

timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
#temporary_df3.to_csv(f'/home/ubuntu/project/4.5HZ/Data/test/test_audio3.csv', index=False)
temporary_df3.to_csv(f'/home/ubuntu/project/4.5HZ/Data/test/API/audio/{year}_{genre}_audio_{timestamp}.csv', index=False)

end_time = time.time()
print(f"작업 소요 시간: {end_time - start_time} 초")

# 2023년 k-pop 10개 뽑는데 대략 10분 * 100분  1000분 16시간???


In [ ]:
start_time = time.time()

artist_cache = {}  

year_list = ['2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010','2009', '2008', '2007', '2006', '2005', '2004', '2003', '2002', '2001', '2000']
genre_list = ['k-pop', 'acoustic', 'club', 'dance', 'edm', 'disco', 'electronic', 'happy', 'hip-hop','party', 'pop', 'r-n-b', 'rock', 'romance', 'sleep', 'soul']

total_workload = len(year_list) * len(genre_list) * 100  
completed_workload = 0

for genre in genre_list:
    # 각 장르별로 데이터를 저장할 빈 DataFrame 생성
    temporary_df = pd.DataFrame()  

    for year in year_list:
        print(f"{year}_{genre}: 다운로드")    
        
        artist_name = []
        track_name = []
        track_popularity = []
        artist_id = []
        track_id = []  
        release_date = []
        album_image = []

        artist_popularity = []
        artist_followers =[]

        track_features_list = []
        
        for i in range(0, 1000, 50):
            track_results = sp.search(q=f'year:{year} genre:{genre}', type='track', limit=50, offset=i)
            for i, t in enumerate(track_results['tracks']['items']):
                artist_id.append(t['artists'][0]['id'])
                artist_name.append(t['artists'][0]['name'])
                track_id.append(t['id'])
                track_name.append(t['name'])
                release_date.append(t['album']['release_date'])
                track_popularity.append(t['popularity'])
                try:
                    album_image.append(t['album']['images'][1]['url'])
                except IndexError:
                    album_image.append(None)                
                print(f"음악트랙 데이터 수집 횟수 : {i+1} // 5초 대기")
                #time.sleep(5)

                
                completed_workload += 1  
                progress = (completed_workload / total_workload) * 100  
                print(f"{year}_{genre}: 진행중") 
                print(f"전체 진행 상황: {progress/10:.2f}% ,,, {progress:.0f}개")

                
        track_df = pd.DataFrame({ 'artist_id': artist_id, 'artist_name': artist_name, 'track_id': track_id, 'track_name': track_name, 
                                 'release_date': release_date, 'track_popularity': track_popularity, 'album_image': album_image})

        
        # 각 연도의 데이터를 temporary_df에 추가
        temporary_df = pd.concat([temporary_df, track_df])

    # 중복 데이터 제거
    temporary_df.drop_duplicates(subset='track_id', inplace=True)

    # 모든 연도를 포함하는 각 장르별로 한 번만 csv 파일로 저장
    timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
    temporary_df.to_csv(f'/home/ubuntu/project/4.5HZ/Data/test/API/track/{genre}_total_track_{timestamp}.csv', index=False)

end_time = time.time()
print(f"작업 소요 시간: {end_time - start_time} 초")


In [ ]:
start_time = time.time()

artist_cache = {}  

year_list = ['2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010','2009', '2008', '2007', '2006', '2005', '2004', '2003', '2002', '2001', '2000']
genre_list = ['disco', 'electronic', 'happy', 'hip-hop','party', 'pop', 'r-n-b', 'rock', 'romance', 'sleep', 'soul']

            # 'edm',

total_workload = len(year_list) * len(genre_list) * 100  
completed_workload = 0

for genre in genre_list:
    # 각 장르별로 데이터를 저장할 빈 DataFrame 생성
    temporary_df2 = pd.DataFrame()  

    for year in year_list:
        print(f"{year}_{genre}: 다운로드")    
        
        artist_name = []
        track_name = []
        track_popularity = []
        artist_id = []
        track_id = []  
        release_date = []
        album_image = []

        artist_popularity = []
        artist_followers =[]

        track_features_list = []
        
        for i in range(0, 1000, 50):
            track_results = sp.search(q=f'year:{year} genre:{genre}', type='track', limit=50, offset=i)
            for i, t in enumerate(track_results['tracks']['items']):
                artist_id.append(t['artists'][0]['id'])
                artist_name.append(t['artists'][0]['name'])
                track_id.append(t['id'])
                # 아티스트 정보를 요청 코드
                artist_info = None
                if t['artists'][0]['id'] in artist_cache:
                    artist_info = artist_cache[t['artists'][0]['id']]
                else:
                    artist_info = sp.artist(t['artists'][0]['id'])
                    artist_cache[t['artists'][0]['id']] = artist_info

                artist_popularity.append(artist_info['popularity'])  
                artist_followers.append(artist_info['followers']['total'])  
                print(f"아티스트 데이터 수집 횟수 : {i+1} // 3초 대기")
                #time.sleep(3)

                completed_workload += 1  
                progress = (completed_workload / total_workload) * 100
                print(f"{year}_{genre}: 진행중")   
                print(f"전체 진행 상황: {progress/10:.2f}% ,,, {progress:.0f}개")
            
        track_df2 = pd.DataFrame({'artist_id': artist_id, 'artist_name': artist_name, 'artist_popularity': artist_popularity, 'artist_followers': artist_followers,
                                  'track_id': track_id})

        temporary_df2 = pd.concat([temporary_df2, track_df2])  

    temporary_df2.drop_duplicates(subset='track_id', inplace=True)
    temporary_df2 = temporary_df2.drop(columns=['track_id'])

    timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
    temporary_df2.to_csv(f'/home/ubuntu/project/4.5HZ/Data/test/API/artist/{genre}_{year}_artist_{timestamp}.csv', index=False)
    #temporary_df2.to_csv(f'./data/{genre}_total_artist_{timestamp}.csv', index=False)

end_time = time.time()
print(f"작업 소요 시간: {end_time - start_time} 초")
